# Smoothing Gyroscope data, handling drift with FFT

## Interactive `bqplot` demo

At this point, we have cleaned the data frame and filtered spikes out of the acceleration data. Now we will perform a *Fast Fourier Transform* which is a mathematical way of mapping the gyroscope signal into frequency space to see the spectral or frequency related characteristics of the wave.

In our analysis we will only look at the real part of the FFTs.

Recall we have three activities to classify from:
* walking
* running
* standing

so we'll be looking at FFT characteristics from both training sets

### Import data

In [1]:
import numpy as np
import pandas as pd
import datetime

import warnings
warnings.filterwarnings("ignore") # ;)

file_path = "./data/Motion_Data_filt_acc.csv"
df = pd.read_csv(file_path)

print("File Read")

File Read


In [2]:
df.head()

,timestamp,time,accelerometer_x,accelerometer_y,accelerometer_z,gyroscope_x,gyroscope_y,gyroscope_z,filtered_accelerometer_x,filtered_accelerometer_y,filtered_accelerometer_z
0,2018-10-24T11:20:00.0000000,0.00,0.046875,-0.008301,1.015625,-1.403809,1.403809,-0.549316,0.044922,-0.003906,1.008301
1,2018-10-24T11:20:00.0100000,0.01,0.049316,-0.003906,1.016602,-1.647949,1.342774,-0.549316,0.045898,-0.008301,1.008301
2,2018-10-24T11:20:00.0200000,0.02,0.045898,-0.010742,1.010254,-2.258301,1.159668,-0.549316,0.046875,-0.010254,1.008301
3,2018-10-24T11:20:00.0300000,0.03,0.044922,-0.010254,1.014160,-2.258301,1.098633,-0.549316,0.047363,-0.010742,1.008301
4,2018-10-24T11:20:00.0400000,0.04,0.050781,-0.011719,1.008301,-2.380371,1.037598,-0.549316,0.047363,-0.011230,1.010254


## Clean / Prep Data

Let's select our relevent data via columns, pandas, and dictionaries! They're helpful for bqplot logic.

First we grab the columns of interest, just their names. The list will be useful for logic later, too.

In [3]:
GYRO_COLUMNS = df.columns[5:8]
print(GYRO_COLUMNS)

Index(['gyroscope_x', 'gyroscope_y', 'gyroscope_z'], dtype='object')


Now we make a new dataframe out of just those columns/series of interest:

In [4]:
dfg = df[['time','gyroscope_x','gyroscope_y','gyroscope_z']]
print(dfg.head())

   time  gyroscope_x  gyroscope_y  gyroscope_z
0  0.00    -1.403809     1.403809    -0.549316
1  0.01    -1.647949     1.342774    -0.549316
2  0.02    -2.258301     1.159668    -0.549316
3  0.03    -2.258301     1.098633    -0.549316
4  0.04    -2.380371     1.037598    -0.549316


Now we can begin some data analysis. We will use `bqplot` interactivity to help with tuning parameters.

In [5]:
import bqplot.pyplot as plt
import bqplot as bq
f = plt.figure()
plt.plot(df['gyroscope_y'],animation_duration=1000)

# uncomment to see only part of the running data
# f.marks[0].scales['x'].max = 45000
# f.marks[0].scales['x'].min = 46000

# uncomment to see only part of the running data
# f.marks[0].scales['x'].max = 25000
# f.marks[0].scales['x'].min = 26000

plt.show()

The first two big blocks of the signal are the walking portion and running portions of the test, respectively. 

We will perform FFTs on each section to try to get an idea of the noise spectrums for each of those activities.

For this, we select a point in the training interval and take a window, preferably of size that is a power of two, and plot the spectral density to look for peaks and noise.

In [6]:
from scipy.fftpack import rfft,rfftfreq, irfft

# window of size of power of 2
# 
window_size = 512

# activity range estimates
walking_start = 20000
walking_end = walking_start + window_size 
running_start = 45000
running_end = running_start + window_size

activities = ['walking','running']
gyro_ffts = {}
signals = {}
x_times = {}
for activity in activities:
    gyro_ffts[activity] = {}
    signals[activity] = {}

x_times.update({'walking': dfg['time'].values[walking_start:walking_end]})
x_times.update({'running': dfg['time'].values[running_start:running_end]})
    
    
for col in GYRO_COLUMNS:
    signals['walking'].update({col: dfg[col].values[walking_start:walking_end]})
    walking_avg = dfg[col].values[walking_start:walking_end].mean()
    
    signals['running'].update({col: dfg[col].values[running_start:running_end]})
    running_avg = dfg[col].values[running_start:running_end].mean()
    
    # We remove the average to remove the "spike at 0",
    # the first FFT coefficient, or the DC bias
    # depending on your perspective of the signal,
    # as a voltage or average of the "acceleration" signal
    
    gyro_ffts['walking'][col] = rfft((signals['walking'][col]-walking_avg))
    gyro_ffts['running'][col] = rfft((signals['running'][col]-running_avg))

    
# d is the sampling frequency, in our case 100hz, so 0.01 seconds
sample = rfftfreq(window_size,d=0.01)
x_freq = sample
print(x_freq[-5:])

[49.609375  49.609375  49.8046875 49.8046875 50.       ]


In [7]:
# quick plot of gyroscope data

fig = plt.figure(title="Gyroscope FFT plot",interacts=pz)



plt.plot(x=x_freq,y=gyro_ffts['running']['gyroscope_z'],opacities=[0.7], colors = ['orangered'],sizes=[0.1])
fig.marks[0].scales['x'].max = 50
fig.marks[0].scales['x'].min = 0
plt.show()


NameError: name 'pz' is not defined

In [8]:
from ipywidgets import (interact, FloatSlider,
                        IntSlider,HBox,VBox,
                        Label, Dropdown)


global filtered_fft
global filt_sig

filter_thresh = IntSlider(value=1000,min=0,max=100000,step=100)
low_pass = FloatSlider(value=10, min=0, max=50, step=0.1)
high_pass = FloatSlider(value=2, min=0, max=50, step=0.1)

    
def filter_and_invert():
    global filtered_fft
    ac = activity_select.value
    gy = gyro_select.value
    # this is a step condition, where if the spectrum meets the condition
    # we allow the fft values to pass, if not, we "squash" them to 0.
    
    filtered_fft = np.where(x_freq<low_pass.value,# condition
                            gyro_ffts[ac][gy],    # value if it survives
                            0)                    # else squashed to 0
    # similar...
    filtered_fft = np.where(x_freq>high_pass.value,filtered_fft,0)
    filtered_fft = np.where(abs(filtered_fft)>filter_thresh.value,filtered_fft,0)
    
    # filter the signal
    global filt_sig
    filt_sig = irfft(filtered_fft)

def low_pee(change):
    filter_and_invert()
    #print('low: ',low_pass.value)
    filt_sig_line.y = filt_sig
    filt_fft_line.y = filtered_fft

def high_pee(change):
    filter_and_invert()
    #print('low: ',low_pass.value)
    filt_sig_line.y = filt_sig
    filt_fft_line.y = filtered_fft
    
def thresh_poo(change):
    filter_and_invert()
    #print("thresh: ",filter_thresh.value)
    filt_sig_line.y = filt_sig
    filt_fft_line.y = filtered_fft

activity_select = Dropdown(options=['walking','running'],value='running')
gyro_select = Dropdown(options=GYRO_COLUMNS,value=GYRO_COLUMNS[0])

def select_gyro(change):
    filter_and_invert()
    filt_sig_line.x = x_times[activity_select.value]
    filt_sig_line.y = filt_sig
    sig_line.y = signals[activity_select.value][change.new]
    sig_line.x = filt_sig_line.x
    filt_fft_line.y = filtered_fft
    
def select_activity(change):
    filter_and_invert()
    filt_sig_line.x = x_times[change.new]
    filt_sig_line.y = filt_sig
    sig_line.y = signals[change.new][gyro_select.value]
    sig_line.x = filt_sig_line.x
    filt_fft_line.y = filtered_fft
    
filter_thresh.observe(thresh_poo,names=['value'])
low_pass.observe(low_pee,names=['value'])
high_pass.observe(high_pee,names=['value'])
gyro_select.observe(select_gyro,names=['value'])
activity_select.observe(select_activity,names=['value'])

x_sc = bq.LinearScale()
xf_sc = bq.LinearScale()
y_lin_sc = bq.LinearScale()
y_log_sc = bq.LinearScale()


gyro_scales = {'x':x_sc, 'y': y_lin_sc}
power_scales = {'x': xf_sc,'y': y_log_sc}

# prepare this bq.Interact object for the signal figure
from bqplot.interacts import PanZoom
pzx = PanZoom(scales={'x':[x_sc]})



filter_and_invert()

# These two lines are plotted together in the same figure:
sig_line = bq.Lines(x=x_times['running'],y=[signals[activity_select.value][gyro_select.value]],scales=gyro_scales,colors=['dodgerblue'],opacities=[0.5])
filt_sig_line = bq.Lines(x=x_times['running'],y=[filt_sig],scales=gyro_scales,colors=['red'],opacities=[0.8])

# This is plotted alone
filt_fft_line = bq.Lines(x=x_freq,y=[filtered_fft],scales=power_scales)


time_ax = bq.Axis(label='time (s)', scale=gyro_scales['x'], marks=[sig_line,filt_sig_line],num_ticks=5)
freq_ax = bq.Axis(label="frequency",scale = bq.LinearScale(max=50,min=0))

gyro_ax = bq.Axis(label='Rotation (deg/s)',orientation='vertical',scale=gyro_scales['y'])
power_ax = bq.Axis(label='magnitude',orientation='vertical',side='left',scale=power_scales['y'])

# figures take in marks, axes, as well as interactions
sig_fig = bq.Figure(marks=[sig_line,filt_sig_line],axes=[time_ax,gyro_ax],interaction=pzx)
fft_fig = bq.Figure(marks=[filt_fft_line],axes=[freq_ax,power_ax])
    
# figure layout options/attribs
fft_fig.layout.width = "300px"
fft_fig.layout.height = "300px"
fft_fig.fig_margin = {'bottom':35,
                    'top':20,
                    'left':50,
                    'right':0}

sig_fig.layout.width = "300px"
sig_fig.layout.height = "300px"
sig_fig.fig_margin = {'bottom':35,
                    'top':20,
                    'left':50,
                    'right':0}

# zoom to smaller portion of data


# Widget box and some layout
slider_label_list = HBox([Label('Power Threshold'),Label('Low pass band'),Label("High Pass band")])
slider_label_list.layout.justify_content = 'space-between'

# Just organizing widgets...
slider_list =  VBox([slider_label_list,HBox([filter_thresh,low_pass,high_pass])])

# Next set of components
signal_dropdowns = VBox([activity_select,gyro_select])

# example layouts, attributes, like children
# helped with aesthetic and spacing

signal_dropdowns.layout.overflow = 'hidden'
signal_dropdowns.layout.width = '275px'
signal_dropdowns.layout.border = 'solid'
signal_dropdowns.children[0].layout.width = '125px'
signal_dropdowns.children[0].layout.border = 'solid'
signal_dropdowns.children[1].layout.width = '125px'
signal_dropdowns.children[1].layout.border = 'solid'



widgets_box = HBox([signal_dropdowns,slider_list])
VBox([widgets_box,HBox([sig_fig,fft_fig])])

When we fine tune parameters for our model, we can import those parameters to use for our ML model. I also want to perform calculus and integration to obtain angle and other measurements! So hopefully this helps removes integration drift and other issues..😬